<a href="https://colab.research.google.com/github/siomaybatagor19/Face_rec.ipynb/blob/main/11209333a_facerecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from PIL import Image
import torch.nn.functional as F

# **Content Initialize**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q /content/drive/MyDrive/imgs.zip

unzip:  cannot find or open /content/drive/MyDrive/imgs.zip, /content/drive/MyDrive/imgs.zip.zip or /content/drive/MyDrive/imgs.zip.ZIP.


# **Using Cuda**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the image paths
trainimgpath = '/content/drive/MyDrive/imgs/Train'
testimgpath = '/content/drive/MyDrive/imgs/Test'

# Define the transformations
transform_train = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Create the datasets
training_set = ImageFolder(root=trainimgpath, transform=transform_train)
test_set = ImageFolder(root=testimgpath, transform=transform_test)

# Create the data loaders
train_loader = DataLoader(training_set, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

# **Get the class indicies**

In [ ]:
# Get the class indices
class_indices = training_set.class_to_idx
face_id = {v: k for k, v in class_indices.items()}
num_classes = len(face_id)

In [ ]:
import pickle
# Save the class indices
with open('face_id.pkl', 'wb') as fs:
    pickle.dump(face_id, fs)

# **Define the model**

In [ ]:
# Define the model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 3)
        self.conv2 = nn.Conv2d(5, 60, 3)
        self.fc1 = nn.Linear(60*14*14, 160)
        self.fc2 = nn.Linear(160, 17)
        self.pool= nn.MaxPool2d(2,2)

    def forward(self, x):
        x = torch.relu(self.pool(self.conv1(x)))
        x = torch.relu(self.pool(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 60*14*14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x
# Create the model and move it to GPU if available
model = ConvNet().to(device)


# **Summary model**

In [ ]:
 summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 5, 62, 62]             140
         MaxPool2d-2            [-1, 5, 31, 31]               0
            Conv2d-3           [-1, 60, 29, 29]           2,760
         MaxPool2d-4           [-1, 60, 14, 14]               0
            Linear-5                  [-1, 160]       1,881,760
            Linear-6                   [-1, 17]           2,737
Total params: 1,887,397
Trainable params: 1,887,397
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.66
Params size (MB): 7.20
Estimated Total Size (MB): 7.91
----------------------------------------------------------------


In [ ]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
start_time = time.time()
model.train()

for epoch in range(30):
    epoch_loss = 0.0
    total_samples = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

    average_epoch_loss = epoch_loss / total_samples
    print(f'Epoch: {epoch + 1}, loss: {average_epoch_loss}')

print('Finished training. Time spent: ', time.time() - start_time)


Epoch: 1, loss: 2.825284744753982
Epoch: 2, loss: 2.3752954078443125
Epoch: 3, loss: 1.5985838310285048
Epoch: 4, loss: 0.6556734801693396
Epoch: 5, loss: 0.2277931457836971
Epoch: 6, loss: 0.07400897883273887
Epoch: 7, loss: 0.029219794577467397
Epoch: 8, loss: 0.018904726068616252
Epoch: 9, loss: 0.020612009276440156
Epoch: 10, loss: 0.008008810134153024
Epoch: 11, loss: 0.005676025087737027
Epoch: 12, loss: 0.004226103576387906
Epoch: 13, loss: 0.004040258605417683
Epoch: 14, loss: 0.0032795627198400766
Epoch: 15, loss: 0.0024139010544758726
Epoch: 16, loss: 0.0020130363537287903
Epoch: 17, loss: 0.0016937879664703469
Epoch: 18, loss: 0.0015611083843837012
Epoch: 19, loss: 0.0012999183842718292
Epoch: 20, loss: 0.0011588060398494374
Epoch: 21, loss: 0.0011185330128655276
Epoch: 22, loss: 0.0009420085844987735
Epoch: 23, loss: 0.0008413200415485666
Epoch: 24, loss: 0.0007479479829381683
Epoch: 25, loss: 0.0006739548379934577
Epoch: 26, loss: 0.000624866506038388
Epoch: 27, loss: 0.00

# **Model Evaluation**

In [ ]:
# Model Evaluation
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on test images: {100 * correct / total}%')


Accuracy of the model on test images: 98.80952380952381%


# **Image Predict**

In [ ]:
# Load an image and predict
def check(impath):
    test_img = Image.open(impath)
    test_img = transform_test(test_img).unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        output = model(test_img)
        _, predicted = torch.max(output.data, 1)
        if face_id[predicted.item()] == 'face17':
            print('Hi, I know you! Your name is ASTRID!! lop yu<3')
        else:
            print('The face is: ', face_id[predicted.item()])

impath = '/content/drive/MyDrive/imgs/Test/face17/3.jpg'
check(impath)

Hi, I know you! Your name is ASTRID!! lop yu<3
